![](https://www.soyhenry.com/_next/static/media/HenryLogo.bb57fd6f.svg)

# Introducción a las bases de datos vectoriales 
## Clase #4 :Rags Y bases de datos Vectoriales 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from dotenv import find_dotenv, load_dotenv
from sklearn.manifold import TSNE
from openai import OpenAI
from langchain_core.documents import Document ### Este es el bueno!!!
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import warnings
warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
movies_info = [
    {
        "title": "RoboCop",
        "content": (
            "RoboCop (1987), dirigida por Paul Verhoeven, es una película de ciencia ficción y acción ambientada en un Detroit "
            "futurista plagado de crimen y controlado por la megacorporación OCP (Omni Consumer Products). "
            "El oficial de policía Alex Murphy (Peter Weller) es brutalmente asesinado por la banda del criminal Clarence Boddicker "
            "(Kurtwood Smith) durante una patrulla. OCP recupera su cuerpo y lo transforma en RoboCop, un cíborg policía "
            "sobrehumano con armadura de titanio, fuerza extraordinaria y puntería perfecta. "
            "Aunque su memoria ha sido oficialmente borrada, Murphy comienza a experimentar fragmentos de recuerdos de su vida "
            "pasada: su esposa Ellen, su hijo Jimmy, y el trauma de su propia muerte. "
            "Mientras lucha contra el crimen callejero, RoboCop descubre una conspiración dentro de OCP liderada por el "
            "vicepresidente Dick Jones, quien está aliado con Boddicker. La película explora temas de identidad, humanidad vs "
            "tecnología, corrupción corporativa, privatización de servicios públicos y la lucha por recuperar la esencia humana "
            "en un mundo dominado por las máquinas y el capitalismo desmedido."
        )
    },
    {
        "title": "The Terminator",
        "content": (
            "The Terminator (1984), dirigida por James Cameron, es una película de ciencia ficción y acción. "
            "En el año 2029, la inteligencia artificial Skynet domina el mundo tras una guerra nuclear que casi extermina "
            "a la humanidad. Para asegurar su victoria, Skynet envía al pasado (1984) un cíborg asesino llamado T-800 "
            "(Arnold Schwarzenegger), un Terminator con apariencia humana pero esqueleto de metal y fuerza sobrehumana. "
            "Su misión es eliminar a Sarah Connor (Linda Hamilton), una joven camarera cuyo hijo aún no nacido, John Connor, "
            "se convertirá en el líder de la resistencia humana. "
            "Kyle Reese (Michael Biehn), un soldado de la resistencia, es enviado al mismo período para protegerla. "
            "La película es una persecución implacable por las calles de Los Ángeles donde Sarah pasa de ser una mujer "
            "ordinaria a una sobreviviente decidida. La película explora temas de destino vs libre albedrío, "
            "los peligros de la inteligencia artificial, paradojas temporales y la resistencia humana ante fuerzas "
            "aparentemente invencibles. Es la primera entrega de una de las franquicias más icónicas del cine de acción."
        )
    },
    {
        "title": "Interstellar",
        "content": (
            "Interstellar (2014), dirigida por Christopher Nolan, es una épica de ciencia ficción protagonizada por "
            "Matthew McConaughey como Cooper, un expiloto de la NASA y granjero. La Tierra se está muriendo: plagas "
            "destruyen los cultivos y tormentas de polvo asfixian a la humanidad. La NASA, operando en secreto, "
            "descubre un agujero de gusano cerca de Saturno que conecta con otra galaxia. "
            "Cooper es reclutado para liderar la misión Endurance junto a la Dra. Amelia Brand (Anne Hathaway), "
            "Romilly (David Gyasi) y Doyle (Wes Bentley). Visitan tres planetas candidatos: Miller (con olas gigantes "
            "donde 1 hora equivale a 7 años terrestres), Mann (un mundo helado donde el Dr. Mann, interpretado por "
            "Matt Damon, los traiciona) y Edmunds. "
            "La película está fundamentada en la física teórica del premio Nobel Kip Thorne y explora la dilatación "
            "temporal, los agujeros negros (el icónico Gargantúa), la quinta dimensión y la gravedad cuántica. "
            "El corazón emocional es la relación entre Cooper y su hija Murph (Jessica Chastain de adulta): "
            "el sacrificio de un padre que envejece más lento que sus hijos debido a la relatividad. "
            "Temas principales: amor trascendiendo el espacio-tiempo, sacrificio, supervivencia de la especie, "
            "y la tensión entre exploración y quedarse con quienes amamos."
        )
    },
    {
        "title": "El Número 23",
        "content": (
            "El Número 23 (2007), dirigida por Joel Schumacher, es un thriller psicológico protagonizado por Jim Carrey "
            "en un papel dramático inusual para él. Walter Sparrow (Carrey) es un empleado de control de animales que "
            "recibe de su esposa Agatha (Virginia Madsen) un misterioso libro titulado 'El Número 23'. "
            "A medida que lee la novela, Walter descubre paralelismos inquietantes entre la vida del protagonista ficticio, "
            "el detective Fingerling, y su propia biografía. Comienza a obsesionarse con el número 23, viéndolo en todas "
            "partes: fechas de nacimiento, direcciones, eventos históricos (cada dato suma 23). "
            "La obsesión lo consume hasta el punto de la paranoia y el insomnio. Su esposa y su hijo Robin intentan "
            "ayudarlo, pero Walter se hunde cada vez más. El giro final revela que Walter mismo escribió el libro años "
            "atrás, durante un episodio traumático que borró de su memoria: un crimen pasional que cometió y que su "
            "mente reprimió como mecanismo de defensa. El libro era su confesión inconsciente. "
            "Temas: obsesión numérica, memoria reprimida, culpa, la línea entre ficción y realidad, y cómo el pasado "
            "que intentamos olvidar siempre encuentra la manera de resurgir."
        )
    },
    {
        "title": "El discreto encanto de la burguesía",
        "content": (
            "El discreto encanto de la burguesía (1972), dirigida por Luis Buñuel, es una comedia surrealista francesa "
            "ganadora del Oscar a Mejor Película Extranjera. El reparto incluye a Fernando Rey, Delphine Seyrig, "
            "Stéphane Audran, Bulle Ogier y Jean-Pierre Cassel. "
            "La premisa es engañosamente simple: un grupo de amigos burgueses intenta repetidamente reunirse para cenar "
            "elegantemente, pero cada intento es frustrado por circunstancias cada vez más absurdas. "
            "Las interrupciones incluyen: llegar al restaurante el día equivocado, una redada militar que convierte "
            "la cena en un ejercicio de combate, descubrir que el dueño del restaurante acaba de morir, ser arrestados "
            "sin razón aparente, y un escenario teatral que los expone como actores sin guion. "
            "La estructura narrativa incluye múltiples capas de sueños dentro de sueños, donde los personajes despiertan "
            "de una pesadilla solo para descubrir que siguen soñando. Escenas recurrentes muestran al grupo caminando "
            "por una carretera vacía sin destino aparente, simbolizando su vacío existencial. "
            "Temas: sátira de la hipocresía burguesa, crítica al colonialismo, la superficialidad de las convenciones "
            "sociales, la naturaleza ilusoria de la realidad, y la futilidad de los rituales sociales de la clase alta."
        )
    },
    {
        "title": "Eterno resplandor de una mente sin recuerdos",
        "content": (
            "Eterno resplandor de una mente sin recuerdos (Eternal Sunshine of the Spotless Mind, 2004), dirigida por "
            "Michel Gondry con guion del aclamado Charlie Kaufman (ganador del Oscar por este guion). "
            "Protagonizada por Jim Carrey como Joel Barish, un hombre introvertido, y Kate Winslet como Clementine "
            "Kruczynski, una mujer impulsiva y de pelo colorido. Tras una dolorosa ruptura, Joel descubre que Clementine "
            "se ha sometido a un procedimiento experimental en la clínica del Dr. Howard Mierzwiak (Tom Wilkinson) "
            "para borrar selectivamente todos los recuerdos de su relación. "
            "Devastado, Joel decide hacerse el mismo procedimiento. La mayor parte de la película transcurre dentro "
            "de la mente de Joel durante el borrado: revivimos su relación en orden inverso, desde las peleas amargas "
            "hasta los momentos más dulces del enamoramiento. A medida que los recuerdos se desvanecen, Joel se da "
            "cuenta de que no quiere olvidarla e intenta esconder a Clementine en rincones ocultos de su memoria "
            "(recuerdos de infancia, humillaciones, memorias no relacionadas). "
            "El elenco secundario incluye a Kirsten Dunst, Mark Ruffalo y Elijah Wood, cada uno con su propia "
            "subtrama sobre la memoria y el arrepentimiento. "
            "Temas principales: la memoria como identidad, el dolor como parte esencial del amor, si borrar el "
            "sufrimiento vale la pena cuando también se pierde la alegría, el determinismo emocional (¿estamos "
            "condenados a repetir los mismos patrones?), y la belleza de los momentos imperfectos."
        )
    }
]

In [ ]:
documents = [
    Document(page_content=m["content"], metadata={"title": m["title"]})
    for m in movies_info
]

In [ ]:
documents

In [ ]:
print("Creando base de datos vectorial...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") # <- - Este es el que hay que cambiar : small, medium, large

# Creamos la base de datos en memoria (Chroma)
vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name="peliculas_collection"
)

In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2}) # Traer solo las 2 más relevantes
llm = ChatOpenAI(model="gpt-4o", temperature=0)
template = """
Eres un experto en cine. Responde la pregunta basándote SOLAMENTE en el siguiente contexto recuperado.
Si la respuesta no está en el contexto.

Para cada película relevante, menciona:
- El nombre de la película
- Cómo se relaciona específicamente con la pregunta
- Detalles concretos del contexto que lo demuestren (director, personajes, trama)

Contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
query = "¿Qué peliculas hablan python?"

## Comparación: LLM sin RAG vs con RAG

Ahora vamos a hacer un experimento clave: **hacerle la misma pregunta al LLM de dos formas distintas**:

1. **Sin RAG (solo títulos):** Le damos al modelo únicamente los **nombres** de las películas de nuestra base de datos, sin descripciones. El modelo debe responder solo con su conocimiento general, que puede ser vago, incorrecto o incompleto sobre estas películas específicas.

2. **Con RAG (contexto recuperado):** El retriever busca los 2 documentos más relevantes en la base vectorial y se los pasa al modelo como contexto. El modelo responde con **detalles específicos** de las descripciones almacenadas.

Observa la diferencia: el RAG permite respuestas **más detalladas y fundamentadas** porque el modelo tiene acceso a la información concreta de nuestra base de datos.

In [ ]:
# Respuesta SIN RAG: el LLM solo recibe los títulos, NO las descripciones
titulos = [m["title"] for m in movies_info]
lista_titulos = ", ".join(titulos)

response_sin_rag = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {"role": "system", "content": (
            "Eres un experto en cine. Responde la pregunta del usuario considerando "
            f"SOLAMENTE estas películas: {lista_titulos}. "
            "No tienes acceso a sinopsis ni descripciones, solo a los títulos."
        )},
        {"role": "user", "content": query}
    ]
)

print("Respuesta SIN RAG (solo títulos, sin descripciones):")
print("-" * 50)
respuesta_sin_rag = response_sin_rag.choices[0].message.content
print(respuesta_sin_rag)

### Ahora con RAG: el modelo recibe contexto de la base de datos vectorial

In [ ]:
respuesta_con_rag = rag_chain.invoke(query)
print("Respuesta CON RAG:")
print("-" * 50)
print(respuesta_con_rag)

#################
## Agentic Rag ##
#################

# Mostrar los documentos recuperados
print("\n\nDocumentos recuperados por el retriever:")
print("=" * 50)
retrieved_docs = retriever.invoke(query)
for i, doc in enumerate(retrieved_docs):
    print(f"\n[Doc {i+1}] {doc.metadata['title']}:")
    print(f"  {doc.page_content[:150]}...")

In [ ]:
# Comparación lado a lado
print("=" * 70)
print("COMPARACIÓN: SIN RAG vs CON RAG")
print("=" * 70)
print(f"\nPregunta: {query}")
print(f"Películas en la base de datos: {lista_titulos}")
print("\n" + "─" * 70)
print("SIN RAG (solo títulos, sin descripciones de nuestra BD):")
print("─" * 70)
print(respuesta_sin_rag)
print("\n" + "─" * 70)
print("CON RAG (con descripciones recuperadas de la BD vectorial):")
print("─" * 70)
print(respuesta_con_rag)
print("\n" + "─" * 70)
print("Documentos recuperados de la base vectorial:")
print("─" * 70)
for doc in retrieved_docs:
    print(f"  -> {doc.metadata['title']}")

## Visualización del Espacio Vectorial

### ¿Qué es un embedding?
Un **embedding** es una representación numérica (vector) de un texto. Cada película y la pregunta se convierten en vectores de alta dimensión (1536 dimensiones con el modelo `text-embedding-3-small`).

### ¿Por qué importa la distancia?
Textos con **significado similar** quedan **cerca** en el espacio vectorial. La **distancia coseno** mide qué tan similares son dos vectores:
- **Distancia 0** = idénticos
- **Distancia ~1** = completamente diferentes

El retriever del RAG selecciona los **k documentos más cercanos** a la pregunta. En el gráfico siguiente:
- Los documentos **recuperados** (más cercanos a la pregunta) se muestran en **verde**
- Los documentos **no recuperados** se muestran en **gris**
- Las **líneas punteadas** muestran la distancia desde la pregunta a cada documento

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

all_docs = vector_db.get()
doc_embeddings = embeddings.embed_documents(all_docs['documents'])
query_embedding = embeddings.embed_query(query)

# Calcular distancias coseno entre la query y cada documento
doc_embeddings_array = np.array(doc_embeddings)
query_embedding_array = np.array(query_embedding).reshape(1, -1)
cosine_dists = cosine_distances(query_embedding_array, doc_embeddings_array)[0]

# Identificar los títulos de los documentos recuperados (k=2)
retrieved_titles = {doc.metadata['title'] for doc in retrieved_docs}
titles = [m['title'] for m in all_docs['metadatas']]

# Reducción dimensional con t-SNE
combined_matrix = np.array(doc_embeddings + [query_embedding])
tsne = TSNE(n_components=2, perplexity=3, random_state=42)
reduced_vecs = tsne.fit_transform(combined_matrix)

doc_coords = reduced_vecs[:-1]
query_coord = reduced_vecs[-1]

# Graficar
fig, ax = plt.subplots(figsize=(12, 8))

for i, title in enumerate(titles):
    is_retrieved = title in retrieved_titles
    color = '#2ecc71' if is_retrieved else '#95a5a6'
    marker_size = 160 if is_retrieved else 100
    edge_color = '#27ae60' if is_retrieved else '#7f8c8d'
    zorder = 5 if is_retrieved else 3

    ax.scatter(doc_coords[i, 0], doc_coords[i, 1],
               c=color, s=marker_size, edgecolors=edge_color, linewidths=2, zorder=zorder)

    # Línea de distancia desde la query al documento
    line_style = '-' if is_retrieved else '--'
    line_alpha = 0.8 if is_retrieved else 0.3
    line_color = '#2ecc71' if is_retrieved else '#bdc3c7'
    ax.plot([query_coord[0], doc_coords[i, 0]],
            [query_coord[1], doc_coords[i, 1]],
            line_style, color=line_color, alpha=line_alpha, linewidth=1.5, zorder=1)

    # Texto de distancia coseno en el punto medio de la línea
    mid_x = (query_coord[0] + doc_coords[i, 0]) / 2
    mid_y = (query_coord[1] + doc_coords[i, 1]) / 2
    dist_text = f"{cosine_dists[i]:.3f}"
    bbox_color = '#d5f5e3' if is_retrieved else '#f2f3f4'
    ax.text(mid_x, mid_y, dist_text, fontsize=8, ha='center', va='center',
            bbox=dict(boxstyle='round,pad=0.2', facecolor=bbox_color, edgecolor='gray', alpha=0.9),
            zorder=6)

    # Etiqueta del título
    label = f"* {title}" if is_retrieved else title
    font_weight = 'bold' if is_retrieved else 'normal'
    ax.text(doc_coords[i, 0] + 0.3, doc_coords[i, 1] + 0.3, label,
            fontsize=9, fontweight=font_weight, zorder=7)

# Query point
ax.scatter(query_coord[0], query_coord[1], c='#e74c3c', s=250, marker='X',
           edgecolors='#c0392b', linewidths=2, zorder=10, label='Tu Pregunta')

ax.set_title(f"Espacio Vectorial: Documentos más cercanos a la pregunta\n\"{query[:60]}...\"",
             fontsize=12, fontweight='bold', pad=15)

# Leyenda personalizada
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='X', color='w', markerfacecolor='#e74c3c',
           markersize=14, markeredgecolor='#c0392b', label='Pregunta (query)'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#2ecc71',
           markersize=10, markeredgecolor='#27ae60', label='Documentos recuperados (k=2)'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#95a5a6',
           markersize=10, markeredgecolor='#7f8c8d', label='Documentos no recuperados'),
    Line2D([0], [0], linestyle='-', color='#2ecc71', alpha=0.8, label='Distancia (recuperado)'),
    Line2D([0], [0], linestyle='--', color='#bdc3c7', alpha=0.5, label='Distancia (no recuperado)'),
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=9, framealpha=0.9)
ax.grid(True, alpha=0.2, linestyle='--')

plt.tight_layout()
plt.show()

# Tabla resumen de distancias
print("\nDistancias coseno (menor = más similar):")
print("=" * 55)
sorted_indices = np.argsort(cosine_dists)
for idx in sorted_indices:
    retrieved_mark = "<- recuperado" if titles[idx] in retrieved_titles else ""
    print(f"  {cosine_dists[idx]:.4f}  |  {titles[idx]:45s} {retrieved_mark}")

---
## Self-Attention: como un Transformer "lee" una frase

Antes de que un texto se convierta en un embedding, el modelo Transformer lo procesa con un mecanismo llamado **self-attention**. Este mecanismo es la razon por la que los embeddings capturan significado semantico y no solo frecuencia de palabras (como TF-IDF).

### Que hace self-attention?

Cuando el modelo lee una frase, **cada palabra mira a todas las demas** para decidir cuales son relevantes para entender su significado en ese contexto:

```
"Tenemos los ojos cansados de tanto ver"

 ojos --atiende--> ver        (alta relevancia: los ojos son los que ven)
 ojos --atiende--> cansados   (alta relevancia: los ojos estan cansados)
 ojos --atiende--> Tenemos    (baja relevancia: palabra funcional)
```

### Por que importa para RAG?

Self-attention es lo que permite que:
- "El felino descansa" y "El gato duerme" tengan embeddings **cercanos** (el modelo entiende que son sinonimos en contexto)
- La busqueda semantica funcione mas alla de coincidencia exacta de palabras
- Los chunks recuperados sean **semanticamente** relevantes, no solo los que comparten keywords

Sin self-attention, los embeddings serian como TF-IDF: bolsas de palabras sin comprension del significado.

### Visualizacion con embeddings de OpenAI

Vamos a simular self-attention usando embeddings de OpenAI: embebemos cada palabra individualmente y medimos la similitud coseno entre todas las palabras de la frase. Esto nos muestra que palabras "se ven" entre si semanticamente, que es exactamente lo que self-attention computa internamente.

In [ ]:
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity as cos_sim

frase = "Tenemos los ojos cansados de tanto ver, y los oidos de tanto oir, y el corazon de tanto odiar."

# Separar en palabras significativas (sin puntuacion ni conectores)
palabras = ["Tenemos", "ojos", "cansados", "tanto", "ver", "oidos", "oir", "corazon", "odiar"]

# Obtener un embedding por cada palabra usando OpenAI
resp = client.embeddings.create(input=palabras, model="text-embedding-3-small")
word_embeddings = np.array([e.embedding for e in resp.data])

# Matriz de similitud coseno entre todas las palabras
sim_matrix = cos_sim(word_embeddings)

print(f'Frase: "{frase}"')
print(f"Palabras embebidas: {palabras}")
print(f"Matriz de similitud: {sim_matrix.shape}")

In [ ]:
# Heatmap de similitud semantica entre palabras (simula self-attention)
fig, ax = plt.subplots(figsize=(10, 8))

# Enmascarar la diagonal (similitud consigo mismo = 1.0, no es informativa)
mask_diag = np.eye(len(palabras), dtype=bool)
sim_display = sim_matrix.copy()
np.fill_diagonal(sim_display, np.nan)

sns.heatmap(
    sim_display,
    xticklabels=palabras,
    yticklabels=palabras,
    annot=True,
    fmt=".2f",
    cmap="YlOrRd",
    ax=ax,
    linewidths=1,
    linecolor="white",
    cbar_kws={"label": "Similitud coseno"},
    mask=mask_diag,
    vmin=0,
    vmax=0.7,
)

ax.set_title(
    'Similitud semantica entre palabras (OpenAI embeddings)\n'
    f'"{frase}"',
    fontsize=12, fontweight="bold", pad=15,
)

plt.xticks(fontsize=11, rotation=45, ha="right")
plt.yticks(fontsize=11, rotation=0)

plt.tight_layout()
plt.show()

print('La frase tiene estructura paralela: ojos/ver, oidos/oir, corazon/odiar.')
print('El heatmap muestra que el modelo captura estas relaciones semanticas.')
print('Las celdas mas oscuras indican palabras con mayor relacion.')

In [ ]:
# Relevance score global: cuanta similitud acumula cada palabra con TODAS las demas?
# Esto simula self-attention: las palabras que mas "conectan" con el resto de la frase
# son las mas importantes para entender el significado completo.

# Sumar similitud con todas las demas palabras (excluyendo consigo misma)
relevance = np.zeros(len(palabras))
for i in range(len(palabras)):
    relevance[i] = sum(sim_matrix[i, j] for j in range(len(palabras)) if j != i)

# Normalizar a [0, 1]
relevance = relevance / relevance.max()

# Ordenar de mayor a menor
orden = np.argsort(relevance)[::-1]
palabras_ord = [palabras[i] for i in orden]
relevance_ord = [relevance[i] for i in orden]

# Colores: la mas importante en rojo fuerte, el resto degradado
colores = []
for i, val in enumerate(relevance_ord):
    if i == 0:
        colores.append("#e74c3c")
    else:
        colores.append(plt.cm.YlOrRd(val * 0.7 + 0.15))

fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(range(len(palabras_ord)), relevance_ord,
               color=colores, edgecolor="black", linewidth=0.5, height=0.7)

ax.set_yticks(range(len(palabras_ord)))
ax.set_yticklabels(palabras_ord, fontsize=13, fontweight="bold")
ax.invert_yaxis()
ax.set_xlabel("Relevance Score (similitud acumulada, normalizada)", fontsize=11)
ax.set_title(
    'Self-Attention: cual es la palabra mas importante de la frase?\n'
    f'"{frase}"',
    fontsize=13, fontweight="bold", pad=12,
)
ax.set_xlim(0, 1.18)
ax.grid(True, axis="x", alpha=0.2, linestyle="--")

# Anotar valores
for i, val in enumerate(relevance_ord):
    label = f"  {val:.3f}"
    if i == 0:
        label += "  <-- mas importante"
    ax.text(val + 0.01, i, label, va="center", fontsize=11,
            fontweight="bold" if i == 0 else "normal",
            color="#e74c3c" if i == 0 else "#333333")

plt.tight_layout()
plt.show()

print(f'La palabra mas importante segun self-attention es: "{palabras_ord[0]}"')
print(f"Esto tiene sentido: \"{palabras_ord[0]}\" es el nucleo emocional de la frase.")
print(f"Las palabras funcionales como \"Tenemos\" y \"tanto\" tienen menor relevancia.")

In [ ]:
# Todas las combinaciones de pares de palabras con su similitud coseno.
# El par con mayor score revela la conexion semantica mas fuerte de la frase.
from itertools import combinations

pares = []
for i, j in combinations(range(len(palabras)), 2):
    pares.append((palabras[i], palabras[j], sim_matrix[i, j]))

# Ordenar de mayor a menor similitud
pares.sort(key=lambda x: x[2], reverse=True)

# Preparar datos para el grafico
labels = [f"{a}  -  {b}" for a, b, _ in pares]
scores = [s for _, _, s in pares]
max_score = scores[0]

# Color: el par top en rojo, top 3 en naranja, el resto en gris
bar_colors = []
for i, s in enumerate(scores):
    if i == 0:
        bar_colors.append("#e74c3c")
    elif i < 3:
        bar_colors.append("#f39c12")
    else:
        bar_colors.append("#bdc3c7")

fig, ax = plt.subplots(figsize=(12, 10))

ax.barh(range(len(pares)), scores,
        color=bar_colors, edgecolor="black", linewidth=0.4, height=0.75)

ax.set_yticks(range(len(pares)))
ax.set_yticklabels(labels, fontsize=11)
ax.invert_yaxis()
ax.set_xlabel("Similitud coseno", fontsize=11)
ax.set_xlim(0, max_score * 1.3)
ax.set_title(
    'Todas las combinaciones de palabras ordenadas por similitud\n'
    f'"{frase}"',
    fontsize=13, fontweight="bold", pad=12,
)
ax.grid(True, axis="x", alpha=0.2, linestyle="--")

# Anotar valores
for i, (a, b, s) in enumerate(pares):
    extra = "  <-- conexion mas fuerte" if i == 0 else ""
    ax.text(s + 0.003, i, f"{s:.3f}{extra}", va="center", fontsize=10,
            fontweight="bold" if i == 0 else "normal",
            color="#e74c3c" if i == 0 else "#555555")

# Leyenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor="#e74c3c", edgecolor="black", label="Par mas fuerte"),
    Patch(facecolor="#f39c12", edgecolor="black", label="Top 3"),
    Patch(facecolor="#bdc3c7", edgecolor="black", label="Resto"),
]
ax.legend(handles=legend_elements, loc="lower right", fontsize=10)

plt.tight_layout()
plt.show()

# Resumen
print("Top 5 conexiones semanticas mas fuertes:")
print("=" * 60)
for a, b, s in pares[:5]:
    print(f'  {s:.3f}  "{a}" <-> "{b}"')

# Contar en cuantos pares top-5 aparece cada palabra
from collections import Counter
top_words = Counter()
for a, b, _ in pares[:5]:
    top_words[a] += 1
    top_words[b] += 1
winner = top_words.most_common(1)[0]
print(f'\nPalabra que domina las conexiones mas fuertes: "{winner[0]}" (aparece en {winner[1]} de los top 5 pares)')

### Multi-Head Attention: multiples perspectivas simultaneas

En un Transformer real, self-attention no se computa una sola vez. Se ejecuta en **multiples cabezas (heads) en paralelo**, cada una aprendiendo a enfocarse en un tipo diferente de relacion:

```
                        Frase de entrada
                              |
              +---------------+---------------+
              |               |               |
          Head 1          Head 2          Head 3
       "Cuerpo"        "Accion"       "Sentimiento"
    ojos <-> oidos    ver <-> oir    cansados <-> odiar
              |               |               |
              +---------------+---------------+
                              |
                     Concatenar + combinar
                              |
                        Embedding final
```

Cada head ve la misma frase pero atiende a relaciones distintas. Luego el modelo combina todas las perspectivas en un solo vector.

Simulemos esto embebiendo cada palabra **en diferentes contextos semanticos** con OpenAI:

In [ ]:
# Simulacion de Multi-Head Attention con OpenAI embeddings.
# Cada "head" embebe las palabras en un contexto semantico diferente,
# lo que cambia las relaciones de similitud entre ellas.

heads = {
    "Head 1: Cuerpo": "parte del cuerpo humano: {}",
    "Head 2: Accion": "accion o verbo que se realiza: {}",
    "Head 3: Sentimiento": "sentimiento o estado emocional: {}",
}

# Embeber cada palabra en cada contexto
head_matrices = {}
for head_name, template in heads.items():
    contextualized = [template.format(p) for p in palabras]
    resp_head = client.embeddings.create(input=contextualized, model="text-embedding-3-small")
    embs = np.array([e.embedding for e in resp_head.data])
    head_matrices[head_name] = cos_sim(embs)

# Visualizar las 3 matrices lado a lado
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

head_colors = ["Blues", "Greens", "Reds"]
for ax, (head_name, matrix), cmap in zip(axes, head_matrices.items(), head_colors):
    mask_diag = np.eye(len(palabras), dtype=bool)
    display = matrix.copy()
    np.fill_diagonal(display, np.nan)

    sns.heatmap(
        display,
        xticklabels=palabras,
        yticklabels=palabras,
        annot=True,
        fmt=".2f",
        cmap=cmap,
        ax=ax,
        linewidths=0.5,
        linecolor="white",
        mask=mask_diag,
        vmin=0.1,
        vmax=0.7,
        cbar=False,
    )
    ax.set_title(head_name, fontsize=12, fontweight="bold")
    plt.sca(ax)
    plt.xticks(fontsize=8, rotation=45, ha="right")
    plt.yticks(fontsize=8, rotation=0)

fig.suptitle(
    'Multi-Head Attention: la misma frase vista desde 3 perspectivas\n'
    f'"{frase}"',
    fontsize=14, fontweight="bold", y=1.04,
)

plt.tight_layout()
plt.show()

# Mostrar el par mas fuerte en cada head
print("Par mas fuerte por cada head:")
print("=" * 60)
for head_name, matrix in head_matrices.items():
    best_score = 0
    best_pair = ("", "")
    for i in range(len(palabras)):
        for j in range(i + 1, len(palabras)):
            if matrix[i, j] > best_score:
                best_score = matrix[i, j]
                best_pair = (palabras[i], palabras[j])
    print(f'  {head_name}:  "{best_pair[0]}" <-> "{best_pair[1]}"  (sim: {best_score:.3f})')

print("\nCada head se especializa en un tipo de relacion diferente.")
print("El Transformer combina todas las perspectivas para generar el embedding final.")